In [21]:
import numpy
import pandas
import scipy.stats as st
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import random
import sys
import numpy
import os
import scipy.stats
import math
import ast
import statsmodels.api as sm

In [22]:
sys.path.append("../../continuous")
sys.path.append("../../continuous/measurements")
sys.path.append("../../utilities")
import ad_marsaglia as ad
from measurements import MEASUREMENTS_CONTINUOUS
import test_anderson_darling
import test_chi_square
import test_kolmogorov_smirnov

In [24]:
from distributions.alpha import ALPHA
from distributions.arcsine import ARCSINE
from distributions.argus import ARGUS
from distributions.beta import BETA
from distributions.beta_prime import BETA_PRIME
from distributions.beta_prime_4p import BETA_PRIME_4P
from distributions.bradford import BRADFORD
from distributions.burr import BURR
from distributions.burr_4p import BURR_4P
from distributions.cauchy import CAUCHY
from distributions.chi_square import CHI_SQUARE
from distributions.chi_square_3p import CHI_SQUARE_3P
from distributions.dagum import DAGUM
from distributions.dagum_4p import DAGUM_4P
from distributions.erlang import ERLANG
from distributions.erlang_3p import ERLANG_3P
from distributions.error_function import ERROR_FUNCTION
from distributions.exponential import EXPONENTIAL
from distributions.exponential_2p import EXPONENTIAL_2P
from distributions.f import F
from distributions.fatigue_life import FATIGUE_LIFE
from distributions.folded_normal import FOLDED_NORMAL
from distributions.frechet import FRECHET
from distributions.f_4p import F_4P
from distributions.gamma import GAMMA
from distributions.gamma_3p import GAMMA_3P
from distributions.generalized_extreme_value import GENERALIZED_EXTREME_VALUE
from distributions.generalized_gamma import GENERALIZED_GAMMA
from distributions.generalized_gamma_4p import GENERALIZED_GAMMA_4P
from distributions.generalized_logistic import GENERALIZED_LOGISTIC
from distributions.generalized_normal import GENERALIZED_NORMAL
from distributions.generalized_pareto import GENERALIZED_PARETO
from distributions.gibrat import GIBRAT
from distributions.gumbel_left import GUMBEL_LEFT
from distributions.gumbel_right import GUMBEL_RIGHT
from distributions.half_normal import HALF_NORMAL
from distributions.hyperbolic_secant import HYPERBOLIC_SECANT
from distributions.inverse_gamma import INVERSE_GAMMA
from distributions.inverse_gamma_3p import INVERSE_GAMMA_3P
from distributions.inverse_gaussian import INVERSE_GAUSSIAN
from distributions.inverse_gaussian_3p import INVERSE_GAUSSIAN_3P
from distributions.johnson_sb import JOHNSON_SB
from distributions.johnson_su import JOHNSON_SU
from distributions.kumaraswamy import KUMARASWAMY
from distributions.laplace import LAPLACE
from distributions.levy import LEVY
from distributions.loggamma import LOGGAMMA
from distributions.logistic import LOGISTIC
from distributions.loglogistic import LOGLOGISTIC
from distributions.loglogistic_3p import LOGLOGISTIC_3P
from distributions.lognormal import LOGNORMAL
from distributions.maxwell import MAXWELL
from distributions.moyal import MOYAL
from distributions.nakagami import NAKAGAMI
from distributions.nc_chi_square import NC_CHI_SQUARE
from distributions.nc_f import NC_F
from distributions.nc_t_student import NC_T_STUDENT
from distributions.normal import NORMAL
from distributions.pareto_first_kind import PARETO_FIRST_KIND
from distributions.pareto_second_kind import PARETO_SECOND_KIND
from distributions.pert import PERT
from distributions.power_function import POWER_FUNCTION
from distributions.rayleigh import RAYLEIGH
from distributions.reciprocal import RECIPROCAL
from distributions.rice import RICE
from distributions.semicircular import SEMICIRCULAR
from distributions.trapezoidal import TRAPEZOIDAL
from distributions.triangular import TRIANGULAR
from distributions.t_student import T_STUDENT
from distributions.t_student_3p import T_STUDENT_3P
from distributions.uniform import UNIFORM
from distributions.weibull import WEIBULL
from distributions.weibull_3p import WEIBULL_3P


In [25]:
_all_distributions = [
    ALPHA,
    ARCSINE,
    ARGUS,
    BETA,
    BETA_PRIME,
    BETA_PRIME_4P,
    BRADFORD,
    BURR,
    BURR_4P,
    CAUCHY,
    CHI_SQUARE,
    CHI_SQUARE_3P,
    DAGUM,
    DAGUM_4P,
    ERLANG,
    ERLANG_3P,
    ERROR_FUNCTION,
    EXPONENTIAL,
    EXPONENTIAL_2P,
    F,
    FATIGUE_LIFE,
    FOLDED_NORMAL,
    FRECHET,
    F_4P,
    GAMMA,
    GAMMA_3P,
    GENERALIZED_EXTREME_VALUE,
    GENERALIZED_GAMMA,
    GENERALIZED_GAMMA_4P,
    GENERALIZED_LOGISTIC,
    GENERALIZED_NORMAL,
    GENERALIZED_PARETO,
    GIBRAT,
    GUMBEL_LEFT,
    GUMBEL_RIGHT,
    HALF_NORMAL,
    HYPERBOLIC_SECANT,
    INVERSE_GAMMA,
    INVERSE_GAMMA_3P,
    INVERSE_GAUSSIAN,
    INVERSE_GAUSSIAN_3P,
    JOHNSON_SB,
    JOHNSON_SU,
    KUMARASWAMY,
    LAPLACE,
    LEVY,
    LOGGAMMA,
    LOGISTIC,
    LOGLOGISTIC,
    LOGLOGISTIC_3P,
    LOGNORMAL,
    MAXWELL,
    MOYAL,
    NAKAGAMI,
    NC_CHI_SQUARE,
    NC_F,
    NC_T_STUDENT,
    NORMAL,
    PARETO_FIRST_KIND,
    PARETO_SECOND_KIND,
    PERT,
    POWER_FUNCTION,
    RAYLEIGH,
    RECIPROCAL,
    RICE,
    SEMICIRCULAR,
    TRAPEZOIDAL,
    TRIANGULAR,
    T_STUDENT,
    T_STUDENT_3P,
    UNIFORM,
    WEIBULL,
    WEIBULL_3P,
]

In [29]:
path = "./data_1000/data_uniform.txt"
sample_distribution_file = open(path, "r")
data = [float(x.replace(",", ".")) for x in sample_distribution_file.read().splitlines()]

In [30]:
# data = pandas.Series(sm.datasets.elnino.load_pandas().data.set_index("YEAR").values.ravel())
# data

In [31]:
measurements = MEASUREMENTS_CONTINUOUS(data)

In [32]:
measurements.num_bins

12

In [33]:
## Calculae Histogram
num_bins = measurements.num_bins
frequencies, bin_edges = numpy.histogram(data, num_bins, density=True)
central_values = [(bin_edges[i] + bin_edges[i + 1]) / 2 for i in range(len(bin_edges) - 1)]


In [34]:
NONE_RESULTS = {
    "test_statistic": None,
    "critical_value": None,
    "p_value": None,
    "rejected": None,
}


In [35]:
RESPONSE = {}
for distribution_class in _all_distributions:
    distribution_name = distribution_class.__name__.lower()
    print(distribution_name)

    validate_estimation = True
    sse = 0
    try:
        distribution = distribution_class(measurements)
        pdf_values = [distribution.pdf(c) for c in central_values]
        sse = numpy.sum(numpy.power(frequencies - pdf_values, 2.0))
    except:
        validate_estimation = False
    print(sse)

    DISTRIBUTION_RESULTS = {}
    v1, v2, v3 = False, False, False
    if validate_estimation and not math.isnan(sse) and not math. isinf(sse):
        try:
            chi2_test = test_chi_square.test_chi_square_continuous(data, distribution, measurements)
            if numpy.isnan(chi2_test["test_statistic"]) == False and math.isinf(chi2_test["test_statistic"]) == False and chi2_test["test_statistic"] > 0:
                DISTRIBUTION_RESULTS["chi_square"] = {
                    "test_statistic": chi2_test["test_statistic"],
                    "critical_value": chi2_test["critical_value"],
                    "p_value": chi2_test["p-value"],
                    "rejected": chi2_test["rejected"],
                }
                v1 = True
            else:
                DISTRIBUTION_RESULTS["chi_square"] = NONE_RESULTS
        except:
            DISTRIBUTION_RESULTS["chi_square"] = NONE_RESULTS

        try:
            ks_test = test_kolmogorov_smirnov.test_kolmogorov_smirnov_continuous(data, distribution, measurements)
            if numpy.isnan(ks_test["test_statistic"]) == False and math.isinf(ks_test["test_statistic"]) == False and ks_test["test_statistic"] > 0:
                DISTRIBUTION_RESULTS["kolmogorov_smirnov"] = {
                    "test_statistic": ks_test["test_statistic"],
                    "critical_value": ks_test["critical_value"],
                    "p_value": ks_test["p-value"],
                    "rejected": ks_test["rejected"],
                }
                v2 = True
            else:
                DISTRIBUTION_RESULTS["anderson_darling"] = NONE_RESULTS
        except:
            DISTRIBUTION_RESULTS["kolmogorov_smirnov"] = NONE_RESULTS
        try:
            ad_test = test_anderson_darling.test_anderson_darling_continuous(data, distribution, measurements)
            if numpy.isnan(ad_test["test_statistic"]) == False and math.isinf(ad_test["test_statistic"]) == False and ad_test["test_statistic"] > 0:
                DISTRIBUTION_RESULTS["anderson_darling"] = {
                    "test_statistic": ad_test["test_statistic"],
                    "critical_value": ad_test["critical_value"],
                    "p_value": ad_test["p-value"],
                    "rejected": ad_test["rejected"],
                }
                v3 = True
            else:
                DISTRIBUTION_RESULTS["anderson_darling"] = NONE_RESULTS
        except:
            DISTRIBUTION_RESULTS["anderson_darling"] = NONE_RESULTS

        print(DISTRIBUTION_RESULTS)
        print(v1, v2, v3)
        if v1 or v2 or v3:
            DISTRIBUTION_RESULTS["sse"] = sse
            DISTRIBUTION_RESULTS["parameters"] = str(distribution.parameters)
            DISTRIBUTION_RESULTS["n_test_passed"] = (
                int(DISTRIBUTION_RESULTS["chi_square"]["rejected"] == False)
                + int(DISTRIBUTION_RESULTS["kolmogorov_smirnov"]["rejected"] == False)
                + int(DISTRIBUTION_RESULTS["anderson_darling"]["rejected"] == False)
            )
            DISTRIBUTION_RESULTS["n_test_null"] = (
                int(DISTRIBUTION_RESULTS["chi_square"]["rejected"] == None)
                + int(DISTRIBUTION_RESULTS["kolmogorov_smirnov"]["rejected"] == None)
                + int(DISTRIBUTION_RESULTS["anderson_darling"]["rejected"] == None)
            )

            RESPONSE[distribution_name] = DISTRIBUTION_RESULTS


alpha
3.0729302103447764e-05


{'chi_square': {'test_statistic': 290.33858058672865, 'critical_value': 15.507313055865453, 'p_value': 0.0, 'rejected': True}, 'kolmogorov_smirnov': {'test_statistic': 0.07019020730840564, 'critical_value': 0.042776500461245, 'p_value': 9.946795255200147e-05, 'rejected': True}, 'anderson_darling': {'test_statistic': 13.340962494679843, 'critical_value': 2.4924360636714047, 'p_value': 6.000056241850515e-07, 'rejected': True}}
True True True
arcsine
1.9837428560801653e-05
{'chi_square': {'test_statistic': 179.8744686170334, 'critical_value': 16.918977604620448, 'p_value': 0.0, 'rejected': True}, 'kolmogorov_smirnov': {'test_statistic': 0.10818843268282197, 'critical_value': 0.042776500461245, 'p_value': 1.1971712510217003e-10, 'rejected': True}, 'anderson_darling': {'test_statistic': 34.18410491684813, 'critical_value': 2.4924360636714047, 'p_value': 5.999999997952088e-07, 'rejected': True}}
True True True
argus
0.0001939270086836214
{'chi_square': {'test_statistic': 177.10548535385138, 

In [36]:
RESPONSE

{'alpha': {'chi_square': {'test_statistic': 290.33858058672865,
   'critical_value': 15.507313055865453,
   'p_value': 0.0,
   'rejected': True},
  'kolmogorov_smirnov': {'test_statistic': 0.07019020730840564,
   'critical_value': 0.042776500461245,
   'p_value': 9.946795255200147e-05,
   'rejected': True},
  'anderson_darling': {'test_statistic': 13.340962494679843,
   'critical_value': 2.4924360636714047,
   'p_value': 6.000056241850515e-07,
   'rejected': True},
  'sse': 3.0729302103447764e-05,
  'parameters': "{'alpha': 12.625000283972085, 'loc': -753.219421291888, 'scale': 11603.542564902025}",
  'n_test_passed': 0,
  'n_test_null': 0},
 'arcsine': {'chi_square': {'test_statistic': 179.8744686170334,
   'critical_value': 16.918977604620448,
   'p_value': 0.0,
   'rejected': True},
  'kolmogorov_smirnov': {'test_statistic': 0.10818843268282197,
   'critical_value': 0.042776500461245,
   'p_value': 1.1971712510217003e-10,
   'rejected': True},
  'anderson_darling': {'test_statistic'

In [39]:
sorted_results_sse = {distribution: results for distribution, results in sorted(RESPONSE.items(), key=lambda x: x[1]["sse"])}

# Mostrar los resultados ordenados
for distribution, results in sorted_results_sse.items():
    print(f"Distribution: {distribution}, SSE: {results['sse']}")


Distribution: bradford, SSE: 1.070358222107061e-06
Distribution: johnson_sb, SSE: 1.382957425092393e-06
Distribution: uniform, SSE: 1.717483917036095e-06
Distribution: generalized_normal, SSE: 1.717483917036383e-06
Distribution: trapezoidal, SSE: 1.7174839293491532e-06
Distribution: semicircular, SSE: 8.785254575214868e-06
Distribution: dagum, SSE: 1.873896147717291e-05
Distribution: arcsine, SSE: 1.9837428560801653e-05
Distribution: weibull, SSE: 2.3980355078125825e-05
Distribution: rayleigh, SSE: 2.4662228185859312e-05
Distribution: rice, SSE: 2.494972094301762e-05
Distribution: maxwell, SSE: 2.614417214111096e-05
Distribution: pert, SSE: 2.7901203927007567e-05
Distribution: chi_square_3p, SSE: 2.7908289614167285e-05
Distribution: f_4p, SSE: 2.8052984514876085e-05
Distribution: folded_normal, SSE: 2.8711293607913532e-05
Distribution: fatigue_life, SSE: 2.9450746289075853e-05
Distribution: inverse_gamma_3p, SSE: 2.986217730941427e-05
Distribution: gamma_3p, SSE: 3.03421434723408e-05
D

In [43]:
aproved_results = {distribution: results for distribution, results in sorted_results_sse.items() if results["n_test_passed"] > 0}

# Mostrar los resultados ordenados
for distribution, results in aproved_results.items():
    print(f"Distribution: {distribution}, SSE: {results['sse']}, Aprobados: {results['n_test_passed']}")

Distribution: bradford, SSE: 1.070358222107061e-06, Aprobados: 3
Distribution: johnson_sb, SSE: 1.382957425092393e-06, Aprobados: 3
Distribution: uniform, SSE: 1.717483917036095e-06, Aprobados: 3
Distribution: trapezoidal, SSE: 1.7174839293491532e-06, Aprobados: 3
